In [ ]:
/// w ##class(DHCExternalService.CardInterface.CardManager).GetPatientInfo(^amy)
ClassMethod GetPatientInfo(json)
{
	/*
	req_info	input			1	查询信息段
	query_key	req_info	C	1		查找方式：2-门诊号 3-住院号 8-身份证号 6-病人id 9-挂号id 10-主页id
	query_content	req_info	C	1		query_key=6、9、10时，该值传病人id
	pvid	req_info	C			query_key=9时，该值传挂号id；query_key=10时，该值传主页id
	*/
	s ^amy = json
	s json = {}.%FromJSON(json)
	s req = json."req_info".%Get(0)
	s key = req."query_key"
	s content = req."query_content"
	s pvid = req."pvid"
	if key = 2 {
		s PatientId = ##Class(DHCWMR.IO.OutService).IGetPatientIDByMrNo(pvid,"O","",.ErrMsg)
	}
	if key = 3 {
		s PatientId = ##Class(DHCWMR.IO.OutService).IGetPatientIDByMrNo(pvid,"I","",.ErrMsg)
	}
	if key = 8 {
		// 身份证号
        if pvid = "" {
            // 如果 pvid 为空，则检查 content
            if content '= ""  {
                s PatientId = $o(^PAPERi("DVA", content, 0))
            } else {
                q "请输入身份证"
            }
        } else {
            // 如果 pvid 不为空，则使用 pvid 查询
            s PatientId = $o(^PAPERi("DVA", pvid, 0))
        }
	}
	if key = 6 {
		;s PatientId = content
		s PatientId = $ORDER(^PAPERi("PAPMI_PatNo",content,0))
	}
	if key = 9 {
		; 挂号id
	}
	if key = 10 {
		; 主页id
	}
	s out = {}
	q:$g(PatientId)="" out.%ToJSON()
	
	s id = PatientId
	s outpno = $p($g(^PAPER(id,"PAT",1)),"^",1)
	s name = $p($g(^PAPER(id,"ALL")),"^",1)
	s sex = $p($g(^PAPER(id,"ALL")),"^",7)
	s:sex'="" sex = $p($g(^CT("SEX",sex)),"^",2)
	s dob = $p($g(^PAPER(id,"ALL")),"^",6)
	s:dob'="" dob = $zd(dob,8)
	s phone = $p($g(^PAPER(id,"PER",1)),"^",11)
	s idcard = $p($g(^PAPER(id,"PAT",3)),"^",6)
	s MaritalDr=$p($g(^PAPER(id,"PER",2)),"^",3)
    if (MaritalDr'="") s MaritalCode=$p($g(^CT("MAR",MaritalDr)),"^",2)
    s home=$p($g(^PAPER(id,"PER",1)),"^",1)
    s eduDr=$P($G(^PAPER(id,"PER",2)),"^",5)
    s:eduDr'="" edu = $p($g(^CT("EDU",eduDr)),"^",2)
    s phone=$p($g(^PAPER(id,"PER",4)),"^",21)   //手机
	s homeTel=$p($g(^PAPER(id,"PER",1)),"^",11)   //家庭电话
    s workTel=$p($g(^PAPER(id,"PER",1)),"^",9)    //工作电话
	s HospId = 2 ;##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
	s age = ##class(web.DHCBillInterface).GetPapmiAge(id,"",HospId)
	s OccupationDR= $p($g(^PAPER(id,"PER",2)),"^",6) ;职业表RowId CTOccupation
	s OccupationCode=""
	s OccupationName=""
	i OccupationDR'="" d
	.s OccupationCode= $p($g(^CT("OCC",OccupationDR)),"^",1) ;职业
	.s OccupationName= $p($g(^CT("OCC",OccupationDR)),"^",2) ;职业
	s SocialStatusDR= $p($g(^PAPER(id,"PER",1)),"^",10) ;病人类型 "PER",1
	s SocialStatusCode= $p($g(^CT("SS",SocialStatusDR)),"^",1) ;病人类型 "PER",1
	s SocialStatusName= $p($g(^CT("SS",SocialStatusDR)),"^",2) ;病人类型 "PER",1
	;国籍/地区代码  国籍/地区名称
	s CountryDr=$p($g(^PAPER(id,"PER",1)),"^",8)
	if (CountryDr'="") {
		s countryCode=$p($g(^CT("COU",CountryDr)),"^",1)
		s countryName=$p($g(^CT("COU",CountryDr)),"^",2)
	}
	;民族代码  民族名称
	s NationDr=$p($g(^PAPER(id,"PER",2)),"^",1)
	if (NationDr'="") {
		s NationCode=$p($g(^CT("NAT",NationDr)),"^",1)
		s NationDesc=$p($g(^CT("NAT",NationDr)),"^",2)
		if ((+NationCode<1)||(+NationCode>56)) {
			s NationCode="97",NationDesc="其他"
		}
	}
	s emp = $p($g(^PAPER(id,"PER",4)),"^",18)
	s PatNokAddress=$p($g(^PAPER(id,"NOK")),"^",7)  //单位地址
	s PatNokPhone=$p($g(^PAPER(id,"NOK")),"^",2)  //单位电话
	s homeTown =$p($g(^PAPER(id,"DHC")),"^",32)
	;s payType = ##class(EMRservice.HISInterface.PatientInfoAssist).PayType(EpisodeID)
	s insuNo = $p($g(^PAPER(id,"ALL")),"^",19)
	s RelationDesc=""
	s RelationID=$p($g(^PAPER(id,"EMP")),"^",4)
	s:RelationID'="" RelationDesc=$P($g(^CT("RLT",RelationID)),"^",2)
	s RelationPhone=$p($g(^PAPER(id,"ALL")),"^",4) 
	s ForeignIDCard=$P($G(^PAPER(id,"PER",2)),"^",4)
	s ForeignAddr = $P($G(^PAPER(id,"PER",1)),"^",1)
	s pat = []
	s item = {}
	s item.pid = outpno ;PatientId
	s item.outpno = outpno
	s item.inpno = ""
	s item."phex_arch_no" = ""
	s item."visit_card_no" = ""
	s item.idno = idcard
	s item."pat_name" = name
	s item."pat_sex" = sex
	s item."pat_age" = age 
	s item."pat_brsdate" = dob
	s item."mari_code" = $g(MaritalCode)
	// 家庭
	s item."pat_home_addr" = home
	s item."pat_home_phno" = homeTel
	s item."pat_home_postcode" = ##Class(web.DHCWMRBaseInfoCtl).GetHomeZip(id)
	// 联系人
	s item."contacts_name" = $p($g(^PAPER(id,"PER",2)),"^",13)
	s item."contacts_relation" = RelationDesc
	s item."contacts_addr" = ForeignAddr
	s item."contacts_phno" = RelationPhone
	s item."pat_contact_idno" = ForeignIDCard
	// 监护人
	s item."grdn_name" = ""
	// 职业
	s item."ocpt_code" = OccupationName
	s item."edu_level_code" = $g(edu)
	s item."nation" = NationDesc
	// 籍贯
	s item."ntvplc" = homeTown
	// 出生地
	s item."pat_baddr" = ##Class(EMRservice.HISInterface.PatientInfoAssist).BirthPlaceNew(id)
	// 国籍
	s item."country_code" = countryName
	// 工作单位
	s item."emp_name" = emp
	s item."pat_emp_phno" = PatNokPhone
	s item."pat_emp_postcode" = ""
	// 户口
	s item."pat_hous_addr" = $p($g(^PAPER(id,"ALL")),"^",19)
	s item."pat_hous_postcode" = $p($g(^PAPER(id,"ALL")),"^",20)
	s item."email" = ""
	s item."pat_qq_no" = ""
	// 患者类型
	s item."pat_type" = SocialStatusName
	// 医疗付款方式
	s item."pay_type" = ""
	/*
	dsctplan_name	pat_info	C			费别
	si_no	pat_info	C			医保号
	si_type_id	pat_info	C			险类
	*/
	s item."dsctplan_name" = ""
	s item."si_no" = insuNo 
	s item."si_type_id" = "" 
	s item."other_card_no" = ""

	d pat.%Push(item)
	s out."pat_info" = pat

	s pv1 = []
	s AdmType = 0
	For {
		s AdmType = $o(^PAPERdr(PatientId,"ADM",AdmType))
		q:AdmType=""
		Set EpisodeID = 0
		for {
		    Set EpisodeID = $o(^PAPERdr(PatientId,"ADM",AdmType,EpisodeID))
		    Quit:EpisodeID=""
		    s item = {}
		    s item."pat_visit_type" = $p($g(^PAPERdr(PatientId,"ADM","O",EpisodeID)),"^",1)
		    s item."pid" = PatientId
		    s item."pvid" = EpisodeID
		    s LocId = $p(^PAADM(EpisodeID),"^",4)
			s Dept = $p($g(^CTLOC(LocId)),"^",2)
			s DeptCode = $p($g(^CTLOC(LocId)),"^",1)
			s AdmWardDr=$p($g(^PAADM(EpisodeID)),"^",70)
			if (AdmWardDr'="") {
				s WardCode = $p($g(^PAWARD(AdmWardDr)),"^",1)
				s Ward = $p($g(^PAWARD(AdmWardDr)),"^",2)
			}
			s Bed = ##class(web.DHCDoc.OP.AjaxInterface).GetPatBedNo($g(EpisodeID))
			s IPTimes=$P(^PAADM(EpisodeID),"^",29)
			s AdmReasonDr=$p(^PAADM(EpisodeID,1),"^",7)
			s AdmReason=$p($g(^PAC("ADMREA",+AdmReasonDr)),"^",2)
			s AdmDate=$p($g(^PAADM(EpisodeID)),"^",6)
			s AdmTime=$p($g(^PAADM(EpisodeID)),"^",7)
			s AdmDateTime = $zd(AdmDate,3)_" "_$zt(AdmTime,1)
			s DischgDate=$p($g(^PAADM(EpisodeID,"DHC")),"^",29)
			s DischgTime=$p($g(^PAADM(EpisodeID,"DHC")),"^",30)
			s DischgDateTime = ""
			if ((DischgDate'="")&&(DischgTime'="")) {
				s DischgDateTime=$zd(DischgDate,3)_" "_$zt(DischgTime,1)
			}
			s AdmNurseInfo=##class(EMRservice.HISInterface.PatientInfoAssist).AdmNurse(EpisodeID,"")
			s nurseId = $p(AdmNurseInfo,"^",2)
			s nurse=$p(AdmNurseInfo,"^",2)
			s docId = $P($g(^PAADM(EpisodeID)),"^",9)
			s docName = ##class(web.INMSurveyComm).GetCareDoctor($g(EpisodeID))
			s regDate = ""
			s regTime = ""
			s que = $o(^User.DHCQueueI("QuePaadmDrIndex",EpisodeID,0))
			if que '= "" {
				s regDate = $lg(^User.DHCQueueD(que),3)
				s:regDate'="" regDate = $zd(regDate,3)
				s regTime = $lg(^User.DHCQueueD(que),20)
				s:regTime'="" regTime = $zt(regTime)
			}
			s regDateTime = regDate _ " " _ regTime
	
		    s item."dept_id" = LocId 
		    s item."dept_name" = Dept
		    s item."inp_ward_id" = AdmWardDr
		    s item."inp_ward_name" = $g(Ward)
		    s item."inp_bed_no" = Bed
		    /*
		    inp_pnurs_id	pv1_info	C	1		责任护士id（住院）
			inp_pnurs	pv1_info	C	1		责任护士（住院）
			pat_atdpscn	pv1_info	C	1		住院医师id
			pat_atdpscn_name	pv1_info	C	1		住院医师
			adta_time	pv1_info	C	1		入院日期
			out_time	pv1_info	C			出院日期
			pat_inp_times	pv1_info	C	1		住院次数（住院）
			pat_diag_desc	pv1_info	C	1		诊断描述
			indept_time	pv1_info	C	1		入科时间（住院）
			nursing_grade	pv1_info	C	1		护理等级
			pat_type	pv1_info	C	1		病人类型
			si_type_id	pv1_info	C	1		险类
			si_type	pv1_info	C	1		医保类型
			arrears	pv1_info	C	1		欠费标记
			balance	pv1_info	C	1		费用余额
			oprtr_id	pv1_info	C	1		操作员编号
			oprtr_name	pv1_info	C	1		操作员姓名
		    */
		    s item."inp_pnurs_id" = nurseId
		    s item."inp_pnurs" = nurseId
		    s item."pat_atdpscn" = docId
		    s item."pat_atdpscn_name" = docName
		    s item."adta_time" = $case(AdmType,"I":AdmDateTime,:regDateTime)
		    s item."out_time" = $case(AdmType,"I":DischgDateTime,:AdmDateTime)
		    s item."pat_inp_times" = IPTimes
		    s item."pat_diag_desc" = ##class(web.DHCDocMain).GetMainDiag(EpisodeID)
		    s item."indept_time" = AdmDateTime
		    s item."nursing_grade" = ##class(web.DHCDocMain).GetNursingRank(EpisodeID)
		    s item."pat_type" = SocialStatusName
		    s item."si_type_id" = AdmReasonDr
		    s item."si_type" = AdmReason
		    s item."balance" = ##class(web.DHCDocOrderCommon).GetCurrentDeposit($g(EpisodeID))
		    s item."arrears" = (item."balance"<0)
		    s item."oprtr_id" = ""
		    s item."oprtr_name" = ""
		    d pv1.%Push(item)
		}
	}
	s out."pv1_info" = pv1

	q out.%ToJSON()
	
	/*
	{
	  "pat_info": [
	    {
	      "pid": 274,
	      "outpno": "1603240010",
	      "inpno": "",
	      "phex_arch_no": "",
	      "visit_card_no": "",
	      "idno": "500108199310060814",
	      "pat_name": "喻坤豪",
	      "pat_sex": "男",
	      "pat_age": "23岁",
	      "pat_brsdate": "19931006",
	      "mari_code": "已婚",
	      "pat_home_addr": "南坪正街115-3-3",
	      "pat_home_phno": "",
	      "pat_home_postcode": "",
	      "contacts_name": "",
	      "contacts_relation": "",
	      "contacts_addr": "",
	      "contacts_phno": "",
	      "pat_contact_idno": "",
	      "grdn_name": "",
	      "ocpt_code": "其他",
	      "edu_level_code": "",
	      "nation": "",
	      "ntvplc": "",
	      "pat_baddr": "",
	      "country_code": "中国",
	      "emp_name": "重庆市轨道交通(集团）有限公司()",
	      "pat_emp_phno": "",
	      "pat_emp_postcode": "",
	      "pat_hous_addr": "",
	      "pat_hous_postcode": "",
	      "email": "",
	      "pat_qq_no": "",
	      "pat_type": "",
	      "pay_type": "商业医疗保险",
	      "dsctplan_name": "普通",
	      "si_no": "C24957234",
	      "si_type_id": "10",
	      "other_card_no": "[就诊卡:23323][IC卡:3232323]"
	    }
	  ],
	  "pv1_info": [
	    {
	      "pat_visit_type": 2,
	      "pid": 34676,
	      "pvid": 1,
	      "dept_id": 37,
	      "dept_name": "康复科病区",
	      "inp_ward_id": "37",
	      "inp_ward_name": "康复科病区",
	      "inp_bed_no": "2",
	      "inp_pnurs_id": "79",
	      "inp_pnurs": "杨冬妮",
	      "pat_atdpscn": "53",
	      "pat_atdpscn_name": "黄萍",
	      "adta_time": "20160826090300",
	      "out_time": "",
	      "pat_inp_times": "1",
	      "pat_diag_desc": "(M51.202)腰椎间盘突出",
	      "indept_time": "20160826090300",
	      "nursing_grade": "Ⅱ级护理",
	      "pat_type": "居民医保",
	      "si_type_id": "10",
	      "si_type": "重庆市金保接口",
	      "arrears": "1",
	      "balance": "4279.75",
	      "oprtr_id": "0094",
	      "oprtr_name": "何婷婷"
	    }
	  ]
	}
	*/
}

/// w ##class(DHCExternalService.CardInterface.CardManager).SaveNewCard(^bob)
ClassMethod SaveNewCard(json)
{
	/*
	{
		"pat_info": [
			{
				"pat_name": "陈静",
				"pat_sex": "女",
				"pat_age": "23",
				"pat_brsdate": "1997-01-01",
				"mari_code": "未婚",
				"pat_home_addr": "重庆",
				"pat_id_num": "500102199612205240",
				"pat_home_phno": "13887888899",
				"ocpt_code": "工人",
				"nation": "汉族",
				"country_code": "中国",
				"cooper_id": ""
			}
		],
		"vcard_info": [
			{
				"pat_card_type": "1",
				"pat_pass_word": "",
				"pat_card_no": "20210001"
			}
		]
	}
	*/
	s ^bob = json
	s json = {}.%FromJSON(json)
	s patInfo = json."pat_info".%Get(0)
	s vcardInfo = json."vcard_info".%Get(0)
	s name = patInfo."pat_name"
	s sex = patInfo."pat_sex"
	s age = patInfo."pat_age"
	s dob = patInfo."pat_brsdate"
	s martial = patInfo."mari_code"
	s home = patInfo."pat_home_addr"
	s idno = patInfo."pat_id_num"
	s phone = patInfo."pat_home_phno"
	s ocpt = patInfo."ocpt_code"
	s nation = patInfo."nation"
	s country = patInfo."country_code"
	s cooper = patInfo."cooper_id"
	
	s cardType = vcardInfo."pat_card_type"
	s pass = vcardInfo."pat_pass_word"
	s cardNo = vcardInfo."pat_card_no"
	s:cardNo'="" cardType = "01"
	/*
	01	条形码
	02	注册医疗卡
	03	医疗卡
	04	人医旧卡
	06	身份证
	05	健康卡
	07	中医院旧卡
	*/
	s occDr=$o(^CT("OCC",0,"Desc",ocpt,0))
   	s:occDr'="" ocpt=$p($g(^CT("OCC",occDr)),"^",1) //职业
   	s sexDr = $o(^CT("SEX",0,"Desc",sex,0))
   	s:sexDr'="" sex = $p($g(^CT("SEX",sexDr)),"^",1)
   	s mariDr = $o(^CT("MAR",0,"Desc",martial,0))
   	s:mariDr'="" martial = $p($g(^CT("MAR",mariDr)),"^",1)
   	s countryDr = $o(^CT("COU",0,"Desc",country,0))
   	s:countryDr'="" country = $p($g(^CT("COU",countryDr)),"^",1) 
   	s nationDr = $o(^CT("NAT",0,"Desc",nation,0))
   	s:nationDr'="" nation = $p($g(^CT("NAT",nationDr)),"^",1)
   	;s ForeignAddress=$p($g(^PAPER(id,"PER",1)),"^",1)
	;s ForeignIDCard=$p($g(^PAPER(id,"PER",2)),"^",4)
	;s ForeignCredType=$p($g(^PAPER(id,"ALL")),"^",25)
   	/*
   	if HisCode=""{
		;直接取本地代码
		s Value=$$ALPHAUP^SSUTIL4(Value)
		if Code="Sex"{
			s HisCode=$O(^CT("SEX",0,"Code",Value,0))
		}elseif Code="IDCardType"{
			s HisCode=$O(^PAC("CARD",0,"Code",Value,0))
		}elseif Code="PatType"{
			s HisCode=$O(^CT("SS",0,"Code",Value,0))
		}elseif Code="Marital"{
			s HisCode=$O(^CT("MAR",0,"Code",Value,0))
		}elseif Code="Nation"{
			s HisCode=$O(^CT("NAT",0,"Code",Value,0))
		}elseif Code="Occupation"{
			s HisCode=$O(^CT("OCC",0,"Code",Value,0))
		}elseif Code="Relation"{
			s HisCode=$O(^CT("RLT",0,"Code",Value,0))
		}elseif Code="Nationality"{
			s HisCode=$O(^CT("COU",0,"Code",Value,0))
		}elseif Code="PayMode"{
			s HisCode=$O(^CT("CTPM",0,"Code",Value,0))
		}else{
			s HisCode=""	
		}
	}
   	*/
	/*
	s sex = $p($g(^PAPER(id,"ALL")),"^",7)
	s:sex'="" sex = $p($g(^CT("SEX",sex)),"^",2)
	s dob = $p($g(^PAPER(id,"ALL")),"^",6)
	s:dob'="" dob = $zd(dob,8)
	s phone = $p($g(^PAPER(id,"PER",1)),"^",11)
	s idcard = $p($g(^PAPER(id,"PAT",3)),"^",6)
	s MaritalDr=$p($g(^PAPER(id,"PER",2)),"^",3)
    if (MaritalDr'="") s MaritalCode=$p($g(^CT("MAR",MaritalDr)),"^",2)
    ;s addr=$p($g(^PAPER(id,"PER",1)),"^",1)
    s EducationDR=$P($G(^PAPER(id,"PER",2)),"^",5)
    s:EducationDR'="" edu = $p($g(^CT("EDU",EducationDR)),"^",2)
	*/
	set writer=##class(%XML.Writer).%New()
    	;set writer.Indent=1
    	;set writer.Charset="UTF-8"

    set status=writer.OutputToString()
    
    set status=writer.StartDocument()
    
        set status=writer.RootElement("Request")
            
        d ..Node(writer,"TradeCode","3014")
        
        d ..Node(writer,"CardTypeCode",cardType)
        d ..Node(writer,"PatientCard",cardNo)
        d ..Node(writer,"SecurityNo","")
        d ..Node(writer,"ChipSerialNo","")
        d ..Node(writer,"PatientType","01")
        ;s name = $p($g(^PAPER(id,"ALL")),"^",1)
        ;s no = $p($g(^PAPER(id,"PAT",1)),"^",1)
        d ..Node(writer,"PatientName",name)
       
        d ..Node(writer,"Sex",sex)
        d ..Node(writer,"DOB",dob)
        ;
        d ..Node(writer,"MaritalStatus",martial)
        // 民族
        d ..Node(writer,"Nation",nation)
        d ..Node(writer,"Occupation",ocpt)
        // 国籍
        d ..Node(writer,"Nationality",country)
        
        ;s idcard = $p($g(^PAPER(id,"PAT",3)),"^",6)
        s idcard = idno
        s idtype = "01"
		
        d ..Node(writer,"IDType",idtype)
        d ..Node(writer,"IDNo",idcard)
        d ..Node(writer,"Address",home)
        d ..Node(writer,"AddressLocus","")
        
        d ..Node(writer,"Zip","")
        d ..Node(writer,"Company","")
        d ..Node(writer,"CompanyAddr","")
        d ..Node(writer,"CompanyZip","")
        d ..Node(writer,"CompanyTelNo","")
        d ..Node(writer,"TelephoneNo","")
        d ..Node(writer,"Mobile",phone)
        d ..Node(writer,"ContactName","")
        d ..Node(writer,"ContactIdNo","")
        
        d ..Node(writer,"ContactAddress","")
        d ..Node(writer,"Relation","")
        d ..Node(writer,"ContactTelNo","")
        d ..Node(writer,"InsureCardFlag","")
        d ..Node(writer,"InsureCardNo","")
        d ..Node(writer,"UserID","wechat001")
        d ..Node(writer,"TerminalID","TD0987654321")
        d ..Node(writer,"TransactionId","1")
        d ..Node(writer,"CardDepositAmt","")
        
        d ..Node(writer,"ChargeDepositAmt","")
        d ..Node(writer,"PayModeCode","2")
        d ..Node(writer,"PayBankCode","")
        d ..Node(writer,"PayCardNo","")
        d ..Node(writer,"PayTradeStr","")
        d ..Node(writer,"AccountPassword","")
        d ..Node(writer,"TheArea","")
        d ..Node(writer,"PatientID","")
        d ..Node(writer,"BankCode","03")
        
        
        d ..Node(writer,"BindFlag","")
        d ..Node(writer,"SignedStatus","")
        d ..Node(writer,"HTBankCardNo","")
        d ..Node(writer,"BankTradeInfo","")
        
        set status=writer.EndRootElement()
        
    set status=writer.EndDocument()
    
    s string=writer.GetXMLString()
    
	s rtn=##class(DHCExternalService.CardInterface.CardManager).SavePatientCard(string)
	
	s out = {}
	s msg = rtn.ResultContent
	if msg'["成功" {
		s out.err = msg
	}
	else {
		; 登记号
		s pid = rtn.PatientID
		s PatId = $ORDER(^PAPERi("PAPMI_PatNo",pid,0))
		
		s cardNo = rtn.PatientCard
		s date = $zdt($h,3)
		s outpno = ""
		s:PatId'="" outpno = $p($g(^PAPER(PatId,"PAT",1)),"^",1)
		
		s vcardInfo = []
		s item = {}
		s item.pid = pid
		s item.outpno = outpno
		s item."pat_card_no" = cardNo
		s item."tran_date" = date
		d vcardInfo.%Push(item)
		
		s out."vcard_info" = vcardInfo
	}
	q out.%ToJSON()
	
	/*
	{
		"vcard_info": [
			{
				"pid": "242892",
				"outpno": "179",
				"pat_card_no": "202100004",
				"tran_date": "20210310184941"
			}
		]
	}
	*/
}

ClassMethod Node(writer, Name, Value)
{
	set status=writer.Element(Name)
    set status=writer.WriteChars(Value)
    set status=writer.EndElement()
}

/// w ##class(DHCExternalService.CardInterface.CardManager).GetPatAdm(^geo)
/// zw ##class(DHCExternalService.CardInterface.CardManager).GetPatAdm("{""req_info"": [{""query_key"": ""6"",""query_content"": ""0000292839"",""pat_visit_type"": """",""pvid"": """",""visit_stat"": """",""begin_time"": ""2024-09-11 00:00:00"",""end_time"": ""2025-03-10 00:00:00""}]}")
/// w ##class(DHCExternalService.CardInterface.CardManager).GetPatAdm("{""req_info"": [{""query_key"": """",""query_content"": """",""pat_visit_type"": ""2"",""pvid"": """",""visit_stat"": """",""begin_time"": ""2024-09-11 00:00:00"",""end_time"": ""2025-03-10 00:00:00"",""pageSize"":""5"",""pageNum"":""2""}]}")
ClassMethod GetPatAdm(json)
{
	/*
	{
	    "req_info": [
	      {
	        "query_key": "6",
	        "query_content": "281833",
	        "pat_visit_type": "1",
	        "pvid": "",
	        "visit_stat": "",
	        "begin_time": "",
	        "end_time": ""
	      }
	    ]
	}
	*/
	s ^geo = json
	s json = {}.%FromJSON(json)
	s req = json."req_info".%Get(0)
	s key = req."query_key"
	s content = req."query_content"
	s visitType = req."pat_visit_type"
	;病人来源|1-查门诊就诊记录;2-查住院就诊记录,为空时查门诊和住院
	s pvid = req."pvid"
	s stat = req."visit_stat"
	s begin = req."begin_time"
	s end = req."end_time"
	s begin = $zdh($p(begin," "),3)
	s end = $zdh($p(end," "),3)
	
	s pageSize = req."pageSize"
	s pageNum = req."pageNum"
	
	s start = ((pageNum - 1) * pageSize) + 1
	s last = (pageNum  * pageSize)
	
	if key = 2 {
		s PatientId = ##Class(DHCWMR.IO.OutService).IGetPatientIDByMrNo(pvid,"O","",.ErrMsg)
	}
	if key = 3 {
		s PatientId = ##Class(DHCWMR.IO.OutService).IGetPatientIDByMrNo(pvid,"I","",.ErrMsg)
	}
	if key = 8 {
		// 身份证号
        if pvid = "" {
            // 如果 pvid 为空，则检查 content
            if content '= ""  {
                s PatientId = $o(^PAPERi("DVA", content, 0))
            } else {
                q "请输入身份证"
            }
        } else {
            // 如果 pvid 不为空，则使用 pvid 查询
            s PatientId = $o(^PAPERi("DVA", pvid, 0))
        }
	}
	if key = 6 {
		s PatientId = $ORDER(^PAPERi("PAPMI_PatNo",content,0))
	}
	if key = 9 {
		; 挂号id
	}
	if key = 10 {
		; 主页id
	}
	s out = {}
	
	s list = []
	// 全院查询
	if ((key = "") && (content = "")) {
		for date=begin:1:end{
			s EpisodeID = 0
			for {
				s EpisodeID=$o(^PAADMi("PAADM_AdmDate",date,EpisodeID))
				q:EpisodeID=""
				s AdmType = $p($g(^PAADM(EpisodeID)),"^",2)
				continue:AdmType="O"&&(visitType=2)
				continue:AdmType="I"&&(visitType=1)
				s sort = $g(sort) + 1
				if ((sort >= start) && (sort <= last)) {
					d OneEpisode
				}
			}
		}
		s rem = sort # pageSize
		s size = ((sort - rem) / pageSize) + 1
		s out."totalCount" = sort
		s out."totalPage" = size
		s out."pat_adt_03" = list
		q out.%ToJSON()
	}
	
	q:PatientId="" out.%ToJSON()
	s list = []
	
	s AdmType = 0
	For {
		s AdmType = $o(^PAPERdr(PatientId,"ADM",AdmType))
		q:AdmType=""
		continue:AdmType'="I"&&(visitType=2)
		continue:AdmType="I"&&(visitType=1)
		s EpisodeID = ""
		for {
		    Set EpisodeID = $o(^PAPERdr(PatientId,"ADM",AdmType,EpisodeID))
		    Quit:EpisodeID=""
		    s AdmDate = $p(^PAADM(EpisodeID),"^",6)
		    continue:((begin'="")&&(AdmDate<begin))
		    continue:((end'="")&&(AdmDate>end))
		    s sort = $g(sort) + 1
			if ((sort >= start) && (sort <= last)) {
				d OneEpisode
			}
		}
	}

	s out."pat_adt_03" = list
	s rem = sort # pageSize
	s size = ((sort - rem) / pageSize) + 1
	s out."totalCount" = sort
	s out."totalPage" = size
	
	q out.%ToJSON()
OneEpisode
	s id = $p($g(^PAADM(EpisodeID)),"^",1)
	s PatientId = id
	s outpno = $p($g(^PAPER(id,"PAT",1)),"^",1)
	s name = $p($g(^PAPER(id,"ALL")),"^",1)
	s sex = $p($g(^PAPER(id,"ALL")),"^",7)
	s:sex'="" sex = $p($g(^CT("SEX",sex)),"^",2)
	s dob = $p($g(^PAPER(id,"ALL")),"^",6)
	s:dob'="" dob = $zd(dob,8)
	s phone = $p($g(^PAPER(id,"PER",1)),"^",11)
	s idcard = $p($g(^PAPER(id,"PAT",3)),"^",6)
	s MaritalDr=$p($g(^PAPER(id,"PER",2)),"^",3)
    if (MaritalDr'="") s MaritalCode=$p($g(^CT("MAR",MaritalDr)),"^",2)
    s addr=$p($g(^PAPER(id,"PER",1)),"^",1)
    s SocialStatusDR= $p($g(^PAPER(id,"PER",1)),"^",10) ;病人类型 "PER",1
	s SocialStatusCode= $p($g(^CT("SS",SocialStatusDR)),"^",1) ;病人类型 "PER",1
	s SocialStatusName= $p($g(^CT("SS",SocialStatusDR)),"^",2) ;病人类型 "PER",1
	s insuNo = $p($g(^PAPER(id,"ALL")),"^",19)
	
	s LocId = $p(^PAADM(EpisodeID),"^",4)
	s Dept = $p($g(^CTLOC(LocId)),"^",2)
	s DeptCode = $p($g(^CTLOC(LocId)),"^",1)
	s AdmWardDr=$p($g(^PAADM(EpisodeID)),"^",70)
	if (AdmWardDr'="") {
		s WardCode = $p($g(^PAWARD(AdmWardDr)),"^",1)
		s Ward = $p($g(^PAWARD(AdmWardDr)),"^",2)
	}
	s Bed = ##class(web.DHCDoc.OP.AjaxInterface).GetPatBedNo($g(EpisodeID))
	s IPTimes=$P(^PAADM(EpisodeID),"^",29)
	s AdmReasonDr=$p(^PAADM(EpisodeID,1),"^",7)
	s AdmReason=$p($g(^PAC("ADMREA",+AdmReasonDr)),"^",2)
	s AdmDate=$p($g(^PAADM(EpisodeID)),"^",6)
	s AdmTime=$p($g(^PAADM(EpisodeID)),"^",7)
	s AdmDateTime = $zd(AdmDate,3)_" "_$zt(AdmTime,1)
	s DischgDate=$p($g(^PAADM(EpisodeID,"DHC")),"^",29)
	s DischgTime=$p($g(^PAADM(EpisodeID,"DHC")),"^",30)
	s DischgDateTime = ""
	if ((DischgDate'="")&&(DischgTime'="")) {
		s DischgDateTime=$zd(DischgDate,3)_" "_$zt(DischgTime,1)
	}
	s AdmNurseInfo=##class(EMRservice.HISInterface.PatientInfoAssist).AdmNurse(EpisodeID,"")
	s nurseId = $p(AdmNurseInfo,"^",2)
	s nurse=$p(AdmNurseInfo,"^",2)
	s HospId = ##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
	s age = ##class(web.DHCBillInterface).GetPapmiAge(id,"",HospId)
	s bah = ##class(web.DHCDocMainOrderInterface).IGetMrNoByEpisodeID(EpisodeID)
	s docId = $P($g(^PAADM(EpisodeID)),"^",9)
	s docName = ##class(web.INMSurveyComm).GetCareDoctor($g(EpisodeID))
	s Info = ##class(web.DHCIPBillDeposit).GetZYChargeInfo(EpisodeID)
	s sumFee = $p(Info,"^",1)
	s prePay = $p(Info,"^",2)
	// Return:	押金余额^押金-自付费用^总费用^自付费用
	// Debug: 	w ##class(web.DHCBillInterface).IGetDepAndBalnace(1628)
	s Info = ##class(web.DHCBillInterface).IGetDepAndBalnace(EpisodeID)
	s balance = $p(Info,"^",1)
	s self = $p(Info,"^",-1)
	s visitStatus = $P($g(^PAADM(EpisodeID)),"^",20)
	s status = 1
	if (visitStatus="D") && (AdmType = "I") {
		s status = 0	
	}
	s regDate = ""
	s regTime = ""
	s que = $o(^User.DHCQueueI("QuePaadmDrIndex",EpisodeID,0))
	if que '= "" {
		s regDate = $lg(^User.DHCQueueD(que),3)
		s:regDate'="" regDate = $zd(regDate,3)
		s regTime = $lg(^User.DHCQueueD(que),20)
		s:regTime'="" regTime = $zt(regTime)
	}
	s regDateTime = regDate _ " " _ regTime
    s item = {}
    s item."pat_visit_type" = visitType
    s item."pid" = outpno
    s item."inpno" = bah
    s item."outno" = outpno
    s item."idno" = idcard
    s item."pat_name" = name
    s item."pat_sex" = sex
    s item."pat_age" = age
    s item."pat_home_addr" = ""
    s item."pat_home_phno" = phone
    s item."pvid" = EpisodeID
    s item."dept_id" = LocId
    s item."dept_name" = Dept
    s item."inp_ward_id" = AdmWardDr
    s item."inp_ward_name" = $g(Ward)
    /*
    inp_pnurs_id	pat_adt_03	C	1		责任护士id（住院特有）
	inp_pnurs	pat_adt_03	C	1		责任护士（（住院特有）
	pat_atdpscn	pat_adt_03	C			住院医师id（门诊为接诊医师id）
	pat_atdpscn_name	pat_adt_03	C	1		住院医师（门诊为接诊医师）
	doctor_name	pat_adt_03	C	1		主治医师（住院特有）
	adta_time	pat_adt_03	C	1		入院日期（门诊为挂号时间）
	out_time	pat_adt_03	C	1		出院日期（门诊为接诊时间）
	indept_time	pat_adt_03	C			入科时间（住院特有）
	nursing_grade	pat_adt_03	C	1		护理等级（住院特有）
	dsctplan_name	pat_adt_03	C	1		费别
	pat_type	pat_adt_03	C	1		病人类型
	si_type_id	pat_adt_03	C	1		险类
	si_type	pat_adt_03	C	1		医保类型
	si_no	pat_adt_03	C			医保号
	out_mask	pat_adt_03	C	1		出院情况（住院特有）
	out_path	pat_adt_03	C	1		出院方式（住院特有）
	oprtr_id	pat_adt_03	C	1		操作员编号（门诊为挂号操作员编号，住院为入院登记操作员编号）
	oprtr_name	pat_adt_03	C	1		操作员姓名（门诊为挂号操作员，住院为入院登记操作员）
	pat_inp_days	pat_adt_03	C	1		住院天数（住院特有）
	charge_sign	pat_adt_03	C	1		结帐标志|1-已结帐（住院特有）
	nobalance_money	pat_adt_03	C	1		总费用
	prepay_money	pat_adt_03	C	1		预交款金额（住院特有）
	guarantee_money	pat_adt_03	C	1		担保金额（住院特有）
	self_payment	pat_adt_03	C			自付金额（住院特有）
	expected_money	pat_adt_03	C			预结费用（住院特有）
	remain_money	pat_adt_03	C	1		患者费用余额（住院特有）
	arrear_flag	pat_adt_03	C	1		欠费标志|1-欠费,0-不欠费（住院特有）
	diet	pat_adt_03	C	1		饮食信息|本次住院最后一次饮食医嘱内容（住院特有）
	allergy	pat_adt_03	C	1		过敏史|1-有过敏史
	status	pat_adt_03	C	1		状态|1-在院（预出院）;0-出院（住院特有）
	visit_status	pat_adt_03	C			就诊状态|门诊：0-等待接诊,1-完成就诊,2-正在就诊,-1标记为不就诊
	receipt_status	pat_adt_03	C			挂号单状态（门诊特有），1-挂号记录;3-被退号记录
    */
    s item."inp_pnurs_id" = nurseId
    s item."inp_pnurs" = nurse
    s item."pat_atdpscn" = docId
    s item."pat_atdpscn_name" = docName
    s item."doctor_name" = docName
    s item."adta_time" = $case(AdmType,"I":AdmDateTime,:regDateTime)
    s item."out_time" = $case(AdmType,"I":DischgDateTime,:AdmDateTime)
    s item."nursing_grade" = ##class(web.DHCDocMain).GetNursingRank(EpisodeID)
    s item."dsctplan_name" = AdmReason
    s item."pat_type" = SocialStatusName
    s item."si_type_id" = AdmReasonDr
    s item."si_type" = AdmReasonDr
    s item."si_no" = insuNo
    s item."out_mask" = ""
    s item."out_path" = ""
    s item."oprtr_id" = ""
    s item."oprtr_name" = ""
    s item."pat_inp_days" = ##class(web.DHCDoc.OP.AjaxInterface).GetInPatAdmDays($g(EpisodeID))
    s item."charge_sign" = ""
    s item."nobalance_money" = sumFee
    s item."prepay_money" = prePay
    s item."guarantee_money" = ""
    s item."expected_money" = ""
    s item."remain_money" = balance
    s item."arrear_flag" = (balance < 0)
    s item."diet" = ""
    s item."allergy" = ""
    s item."status" = status
    s item."visit_status" = 1
    s item."receipt_status" = ""

    s arr = []
    s MRAdm=$p($g(^PAADM(EpisodeID)),"^",61)
    s Diag = 0
    for {
        Set Diag=$O(^MR(MRAdm,"DIA",Diag))
        Quit:(Diag="")
        
        s doctor=$p($g(^MR(MRAdm,"DIA",Diag)),"^",4)
        s:doctor'="" doctor=$P($G(^CTPCP(doctor,1)),"^",2)
		s icd=$p($g(^MR(MRAdm,"DIA",Diag)),"^",1)
		s icdCode = $p($g(^MRC("ID",icd)),"^",1)
		s icdDesc = $p($g(^MRC("ID",icd)),"^",2)
		s diagDate=$p($g(^MR(MRAdm,"DIA",Diag)),"^",7)
		s:diagDate'="" diagDate = $zd(diagDate,3)
		s diagTime=$p($g(^MR(MRAdm,"DIA",Diag)),"^",8)
		s:diagTime'="" diagTime = $zt(diagTime)
		s diagDateTime = diagDate _ " " _ diagTime
        s diag = {}
        /*
        dz_source	dg1_info	C	1		记录来源
		dzsim_type	dg1_info	C	1		诊断类型
		diag_serial	dg1_info	C	1		诊断次序
		dz_content	dg1_info	C	1		诊断描述
		dz_code	dg1_info	C	1		疾病编码
		dz_name	dg1_info	C	1		疾病名称
		dz_type	dg1_info	C	1		疾病类别
		diag_status_code	dg1_info	C	1		1-疑诊
		cnd_outcm_name	dg1_info	C			出院情况
		adta_cnd_name	dg1_info	C			入院病情
		diag_recoder	dg1_info	C	1		记录人
		diag_date	dg1_info	C	1		记录日期
        */
        s diag."id" = MRAdm_Diag
        s diag."dz_source" = ""
        s diag."dzsim_type" = ""
        s diag."diag_serial" = Diag
        s diag."dz_content" = "("_icdCode_")"_icdDesc
        s diag."dz_code" = icdCode
        s diag."dz_name" = icdDesc
        s diag."dz_type" = ""
        s diag."diag_status_code" = 0
        s diag."cnd_outcm_name" = ""
        s diag."adta_cnd_name" = ""
        s diag."diag_recoder" = doctor
        s diag."diag_date" = diagDateTime

        d arr.%Push(diag)
    }
    s item."dg1_info" = arr

    d list.%Push(item)
	/*
	{
		"pat_adt_03": [
		{
		  "pid": "281833",
		  "inpno": "15020646",
		  "outno": "33710",
		  "idno": "512301196309143264",
		  "pat_name": "艾玉琴",
		  "pat_sex": "女",
		  "pat_age": "52岁",
		  "pat_home_addr": "龙桥街道办事处石塔村5社",
		  "pat_home_phno": "",
		  "pvid": "1",
		  "dept_id": "514",
		  "dept_name": "【总】骨科",
		  "inp_ward_id": "514",
		  "inp_ward_name": "【总】骨科",
		  "inp_pnurs": "赵安然",
		  "pat_atdpscn_name": "桂震",
		  "adta_time": "2015-02-23 15:00:49",
		  "out_time": "2016-01-11 14:06:01",
		  "nursing_grade": "Ⅱ级护理",
		  "dsctplan_name": "普通",
		  "pat_type": "居民医保",
		  "si_type_id": "10",
		  "si_type": "重庆市金保接口",
		  "si_no": "B81572106",
		  "out_mask": "治愈",
		  "out_path": "正常",
		  "oprtr_id": "000450",
		  "oprtr_name": "梁春梅",
		  "pat_inp_days": "322",
		  "charge_sign": "1",
		  "nobalance_money": "88850.87",
		  "prepay_money": "1000",
		  "guarantee_money": "0",
		  "expected_money": "0",
		  "remain_money": "-87850.87",
		  "arrear_flag": "-1",
		  "diet": "糖尿病普食",
		  "allergy": "0",
		  "status": "0",
		  "dg1_info": [
		    {
		      "id": "475063",
		      "dz_source": "2",
		      "dzsim_type": "1",
		      "diag_serial": "1",
		      "dz_content": "(E21.005)上肢骨囊性纤维性骨炎",
		      "dz_code": "E21.005",
		      "dz_name": "上肢骨囊性纤维性骨炎",
		      "dz_type": "D",
		      "diag_status_code": "",
		      "cnd_outcm_name": "",
		      "adta_cnd_name": "",
		      "diag_recoder": "梁春梅",
		      "diag_date": "2015-02-23 15:02:08"
		    }, {
		      "id": "476228",
		      "dz_source": "2",
		      "dzsim_type": "2",
		      "diag_serial": "1",
		      "dz_content": "(E21.005)上肢骨囊性纤维性骨炎",
		      "dz_code": "E21.005",
		      "dz_name": "上肢骨囊性纤维性骨炎",
		      "dz_type": "D",
		      "diag_status_code": "",
		      "cnd_outcm_name": "",
		      "adta_cnd_name": "",
		      "diag_recoder": "系统管理员",
		      "diag_date": "2016-01-11 14:07:49"
		    }
		  ]
		}
		]
	}
*/
}

/// w ##class(DHCExternalService.CardInterface.CardManager).GetPatOrd(^ali)
/// w ##class(DHCExternalService.CardInterface.CardManager).GetPatOrd("{""req_info"":[{""pid"":""0000330118"",""pvid"":""1"",""pat_visit_type"":""2""}]}")
ClassMethod GetPatOrd(json)
{
	/*
	{
	  "req_info": [
	    {
	      "pid": "242282",
	      "pvid": "1"
	    }
	  ]
	}
	*/
	s ^ali=json
	s json = {}.%FromJSON(json)
	s req = json."req_info".%Get(0)
	s pid = req."pid"
	s pvid = req."pvid"
	s visitType = req."pat_visit_type"
	s PatientID = $ORDER(^PAPERi("PAPMI_PatNo",pid,0))
	s out = {}
	q:PatientID="" out.%ToJSON()
	s list = []

	s outpno = $p($g(^PAPER(PatientID,"PAT",1)),"^",1)
	Set AdmType = 0
	For {
	    Set AdmType = $o(^PAPERdr(PatientID,"ADM",AdmType))
	    Quit:AdmType=""
	    continue:((AdmType'="I")&&(visitType=2))
		continue:((AdmType="I")&&(visitType=1))
	    Set EpisodeID = 0
	    For {
	        Set EpisodeID = $o(^PAPERdr(PatientID,"ADM",AdmType,EpisodeID))
	        Quit:EpisodeID=""
	        
	        Set ord = 0
	        For {
	            Set ord = $o(^OEORD(0,"Adm",EpisodeID,ord))
	            Quit:ord=""
	            Set item = 0
	            For {
	                Set item = $o(^OEORD(ord,"I",item))
	                Quit:item=""
	                s stat=$p($g(^OEORD(ord,"I",item,1)),"^",13)
	                continue:stat=""
	                s statCode = $p(^OEC("OSTAT",stat),"^",1)
	                s statDesc = $p(^OEC("OSTAT",stat),"^",2)
	                ;continue:'$lf($lb("V","E","D","U"),statCode)
	                // 1-新开；2-校对疑问；3-已校对；4-已作废；5-已重整；6-已暂停；7-已启用；8-已停止；9-已确认停止
	                // 1 4 8
	                s status = $case(statCode,"V":1,"E":3,"U":4,"D":8,:1)
	                s arcim = $p($g(^OEORD(ord,"I",item,1)),"^",2)
	                s arcimDesc = $p($g(^ARCIM(+arcim,1,1)),"^",2)
	                s dept = $p($g(^OEORD(ord,"I",item,1)),"^",3)
	                s deptDesc = ""
	                s:dept'="" deptDesc = $p($g(^CTLOC(dept)),"^",2)
	                s doctor = $p($g(^OEORD(ord,"I",item,1)),"^",11)
	                s:doctor'="" doctor=$P($G(^CTPCP(doctor,1)),"^",2)
	                s spec = ""
	                s freq = $p($g(^OEORD(ord,"I",item,2)),"^",4)
	                s:freq'="" freq = $p($g(^PHCFR(freq)),"^",2)
	                s date = $p($g(^OEORD(ord,"I",item,3)),"^",7)
	                s:date'="" date = $zd(date,3)
	                s time = $p($g(^OEORD(ord,"I",item,3)),"^",15)
	                s:time'="" time = $zt(time)
	                s dateTime = date _ " " _time
	                // 处理
	                s seeUser = $p($g(^OEORD(ord,"I",item,"NUR")),"^",4)
	                s:seeUser'="" seeUser = $p($g(^SSU("SSUSR",seeUser)),"^",2)
	                s seeDate = $p($g(^OEORD(ord,"I",item,"NUR")),"^",5)
	                s:seeDate'="" seeDate = $zd(seeDate,3)
	                s seeTime = $p($g(^OEORD(ord,"I",item,"NUR")),"^",6)
	                s:seeTime'="" seeTime = $zt(seeTime)
	                s seeDateTime = seeDate _ " " _seeTime
	                // 停止处理
	                s stopSeeUser = $p($g(^OEORD(ord,"I",item,"NUR")),"^",1)
	                s:stopSeeUser'="" stopSeeUser = $p($g(^SSU("SSUSR",stopSeeUser)),"^",2)
	                s stopSeeDate = $p($g(^OEORD(ord,"I",item,"NUR")),"^",2)
	                s:stopSeeDate'="" stopSeeDate = $zd(stopSeeDate,3)
	                s stopSeeTime = $p($g(^OEORD(ord,"I",item,"NUR")),"^",3)
	                s:stopSeeTime'="" stopSeeTime = $zt(stopSeeTime)
	                s stopSeeDateTime = stopSeeDate _ " " _ stopSeeTime
	                // 停止
	                s stopDoctor = $p($g(^OEORD(ord,"I",item,3)),"^",29)
	                s:stopDoctor'="" stopDoctor = $p($g(^CTPCP(stopDoctor)),"^",2)
	                s stopDate = $p($g(^OEORD(ord,"I",item,3)),"^",34)
	                s:stopDate'="" stopDate = $zd(stopDate,3)
	                s stopTime = $p($g(^OEORD(ord,"I",item,2)),"^",15)
	                s:stopTime'="" stopTime = $zt(stopTime)
	                s stopDateTime = stopDate _ " " _ stopTime
	                // update
	                s updateDate = $p($g(^OEORD(ord,"I",item,11)),"^",4)
	                s:updateDate'="" updateDate = $zd(updateDate,3)
	                s updateTime = $p($g(^OEORD(ord,"I",item,11)),"^",5)
	                s:updateTime'="" updateTime = $zt(updateTime)
	                s updateDateTime = updateDate _ " " _ updateTime
	                // start
	                s startDate = $p($g(^OEORD(ord,"I",item,1)),"^",9)
	                s:startDate'="" startDate = $zd(startDate,3)
	                s startTime = $p($g(^OEORD(ord,"I",item,1)),"^",10)
	                s:startTime'="" startTime = $zt(startTime)
	                s startDateTime = startDate _ " " _ startTime
	                // end
	                s endDate = $p($g(^OEORD(ord,"I",item,9)),"^",9)
	                s:endDate'="" endDate = $zd(endDate,3)
	                s endTime = $p($g(^OEORD(ord,"I",item,9)),"^",10)
	                s:endTime'="" endTime = $zt(endTime)
	                s endDateTime = endDate _ " " _ endTime
	                s packqty = $p($g(^OEORD(ord,"I",item,4)),"^",9)
	                s doseqty = $p($g(^OEORD(ord,"I",item,2)),"^",1)
	                s doseUom = $p($g(^OEORD(ord,"I",item,2)),"^",3)
	                s:doseUom'="" doseUom = $p($g(^CT("UOM",doseUom)),"^",2)
	                s depProcNotes = ""  ;备注 OEORI_DepProcNotes
				    s depProcNotesIndex = 0
				    f  s depProcNotesIndex=$o(^OEORD(ord,"I",item,"DEP",depProcNotesIndex)) q:depProcNotesIndex=""  d
				    .q:(^OEORD(ord,"I",item,"DEP",depProcNotesIndex)="")
				    .s depProcNotes = depProcNotes_","_^OEORD(ord,"I",item,"DEP",depProcNotesIndex)
				    ; 0-无执行的叮嘱；1-病人所在科室；2-病人所在病区；3-开单人所在科室；4-指定科室;5-院外执行；药品医嘱界面表现为正常、外购药、出院带药。
				    s nature = 2
				    ; 4-卫材,5-西成药,6-中成药,7-中草药,8-配方,9-成套,C-检验,D-检查,E-治疗,F-手术,G-麻醉,H-护理,I-膳食,K-输血,L-输氧,M-材料,Z-其他
				   	
				    s subCat = $p($g(^ARCIM(+arcim,1,1)),"^",10)
				    s cat = $p($g(^ARC("IC",subCat)),"^",8)
				    s catCode = $p($g(^OEC("ORCAT",cat)),"^",1)
				    s type = ##class(web.DHCDocOrderCommon).GetOrderType(arcim)
				    s type = $case(type,"R":5,"X":"D","L":"C","N":"E",:"Z")
				    s type = $case(catCode,"西药":1,"中成药":5,"中草药":7,"医嘱套":9,"手术费":"F","手术麻醉":"G","护理":"H","检查":"D",:type)
				    ; 0-长期;1-临时
				    s prior = $p($g(^OEORD(ord,"I",item,1)),"^",8)
				    if ##class(appcom.OEOrdItem).ISLongOrderPrior(prior) {
						s prior = 0    
					}
					else {
						s prior = 1	
					}
	                s it = {}
	                s it."pat_visit_type" = $case(AdmType,"I":1,:2)
	                s it."pid" = outpno
	                s it."pvid" = EpisodeID
	                s it."order_id" = ord
	                /*
	                order_group_sno	advice_list	C	1	　	相关id
					nb_sno	advice_list	C	　	　	婴儿
					order_expidate_type	advice_list	C	1	　	医嘱期效|0-长期;1-临时
					citem_type	advice_list	C	1	　	诊疗类别|4-卫材,5-西成药,6-中成药,7-中草药,8-配方,9-成套,C-检验,D-检查,E-治疗,F-手术,G-麻醉,H-护理,I-膳食,K-输血,L-输氧,M-材料,Z-其他
					citem_id	advice_list	C	1	　	诊疗项目id
					citem_name	advice_list	C	1	　	诊疗项目名称
					order_content	advice_list	C	1	　	医嘱内容
					order_drask	advice_list	C	　	　	医生嘱托
					order_once_qunt	advice_list	C	1	　	单次用量
					order_total_qunt	advice_list	C	1	　	总给予量
					order_once_qunt_unit	advice_list	C	1	　	剂量单位
					order_freq_name	advice_list	C	1	　	执行频次
					emg_sign	advice_list	C	1	　	紧急标志
					order_status	advice_list	C	1	　	医嘱状态|-1-未生效的暂存医嘱；1-新开；2-校对疑问；3-已校对；4-已作废；5-已重整；6-已暂停；7-已启用；8-已停止；9-已确认停止
					order_exe_time_start	advice_list	C	　	　	开始执行时间|yyyy-MM-dd HH:mm:ss
					order_exe_time_end	advice_list	C	　	　	执行终止时间|yyyy-MM-dd HH:mm:ss
					apply_dept_id	advice_list	C	1	　	开嘱科室id
					apply_dept_name	advice_list	C	1	　	开嘱科室
					placer_name	advice_list	C	1	　	开嘱医生
					apply_time	advice_list	C	1	　	开嘱时间|yyyy-MM-dd HH:mm:ss
					chk_time	advice_list	C	　	　	校对时间|yyyy-MM-dd HH:mm:ss
					order_chk_nurse	advice_list	C	　	　	校对护士
					orderstp_time	advice_list	C	　	　	停嘱时间|yyyy-MM-dd HH:mm:ss
					orderstp_placer	advice_list	C	　	　	停嘱医生
					conterm_nurse	advice_list	C	　	　	确认停嘱护士
					conterm_time	advice_list	C	　	　	确认停嘱时间|yyyy-MM-dd HH:mm:ss
					operate_time	advice_list	C	　	　	医嘱操作时间|yyyy-MM-dd HH:mm:ss
					exe_nature	advice_list	C	　	　	执行性质|0-无执行的叮嘱；1-病人所在科室；2-病人所在病区；3-开单人所在科室；4-指定科室;5-院外执行；药品医嘱界面表现为正常、外购药、出院带药。
	                */
	                s it."order_group_sno" = $p($g(^OEORD(ord,"I",item,1)),"^",1)
	                s it."nb_sno" = 0
	                s it."order_expidate_type" = prior
	                s it."citem_type" = type
	                s it."citem_id" = +arcim
	                s it."citem_name" = arcimDesc
	                s it."order_content" = arcimDesc
	                s it."order_drask" = depProcNotes
	                s it."order_once_qunt" = doseqty
	                s it."order_total_qunt" = packqty
	                s it."order_once_qunt_unit" = doseUom
	                s it."order_freq_name" = freq
	                s it."emg_sign" = 0
	                s it."order_status" =  status ;$p($g(^OEORD(ord,"I",item,1)),"^",15)
	                s it."order_exe_time_start" = startDateTime
	                s it."order_exe_time_end" = endDateTime
	                s it."apply_dept_id" = dept
	                s it."apply_dept_name" = deptDesc
	                s it."placer_name" = doctor
	                s it."apply_time" = dateTime
	                s it."chk_time" = seeDateTime
	                s it."order_chk_nurse" = seeUser
	                s it."orderstp_time" = stopDateTime
	                s it."orderstp_placer" = stopDoctor
	                s it."conterm_nurse" = stopSeeUser
	                s it."conterm_time" = stopSeeDateTime
	                s it."operate_time" = updateDateTime
	                s it."exe_nature" = ""

	                d list.%Push(it)
	            }
	        }
	    }
	}

	s out."advice_list" = list

	q out.%ToJSON()
	/*
	{
	  "advice_list": [
	    {
	      "pat_visit_type": 2,
	      "pid": 242282,
	      "pvid": 1,
	      "order_id": 6371,
	      "order_group_sno": 6372,
	      "nb_sno": 0,
	      "order_expidate_type": 0,
	      "citem_type": "6",
	      "citem_id": 6589,
	      "citem_name": "天麻素注射液",
	      "order_content": "天麻素注射液 200mg:2ml/支",
	      "order_drask": null,
	      "order_once_qunt": 600,
	      "order_total_qunt": null,
	      "order_once_qunt_unit": "支",
	      "order_freq_name": "每天上午一次",
	      "emg_sign": 0,
	      "order_status": 9,
	      "order_exe_time_start": "2014-11-01 09:11:00",
	      "order_exe_time_end": "2014-11-02 10:11:00",
	      "apply_dept_id": 76,
	      "apply_dept_name": "老年病科",
	      "placer_name": "陈正权",
	      "apply_time": "2014-11-01 09:11:00",
	      "chk_time": "2014-11-01 09:11:00",
	      "order_chk_nurse": "黄银",
	      "orderstp_time": "2014-11-02 08:11:22",
	      "orderstp_placer": "陈正权",
	      "conterm_nurse": "黄银",
	      "conterm_time": "2014-11-02 10:11:00",
	      "operate_time": null,
	      "exe_nature": 4
	    }
	  ]
	}
	*/
}
